# Clustering Food Venues in Canadian Cities

This notebook will explore postal codes and food venues in Canada's three largest cities: Toronto, Montreal and Calgary.

Our objective is to visualize where different categories of food venues are clustered, which may have some implications for the surrounding population demographics and business environment.

(Note: Forward Sortation Areas (FSA) are the first three letters of Canadian Postal Codes. But here we use the term interchangeably with Postal Code or Post Code.)

Import everything we need.

In [1]:
try: 
    import dill
except: 
    !conda install -c conda-forge dill --yes # install only if necessary
    import dill
# import libraries

# time for performance measures
import time

# numpy for data vectors
import numpy as np

# pandas for data analysis
import pandas as pd
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

from collections import Counter

# # transform json into pandas dataframe
# from pandas.io.json import json_normalize

# json
import json

# requests
import requests

# nominatim to convert an address into latitude and longitude values
try:
    from geopy.geocoders import Nominatim 
except (ImportError, ModuleNotFoundError): #install only if necessary
    !conda install -c conda-forge geopy --yes 
    from geopy.geocoders import Nominatim

# Matplotlib for plotting
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as py

# import k-means from sklearn clustering
from sklearn.cluster import KMeans

# folium for visualizaing maps
try:
    import folium
except (ImportError, ModuleNotFoundError): #install only if necessary
    !conda install -c conda-forge folium=0.5.0 --yes
    import folium # plotting library
from folium.plugins import MarkerCluster    
from folium.plugins import FastMarkerCluster

Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - dill


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.9.0                |   py38h5347e94_1         3.0 MB  conda-forge
    dill-0.3.2                 |     pyh9f0ad1d_0          59 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be INSTALLED:

  dill               conda-forge/noarch::dill-0.3.2-pyh9f0ad1d_0

The following packages will be UPDATED:

  conda                                4.8.5-py38h5347e94_3 --> 4.9.0-py38h5347e94_1



conda-4.9.0          | 3.0 MB    | ##################################### | 100% 
dill-0.3.2           | 59 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying tr

# Foursquare Data

Define Foursquare Credentials and Version
(credentials removed in public copy)

In [4]:
CLIENT_ID = 'OIIYCWYJJVFDXLWUSTBIUE5IYMLKIMWPG0EADSI4CLO3LFPK' # your Foursquare ID
CLIENT_SECRET = 'MUKQKAHXLTOSURSIWHMMZM4PDRYZT5AMKRSABL3HN5YQQ33M' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

# Postal Code Data

I am using wikipedia tables of postal codes in Canada:

Toronto: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

Montreal: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_H

Calgary: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_T

In [2]:
city_codes = pd.DataFrame([['M','Toronto, Ontario'],['H','Montreal, Quebec'],['T', 'Calgary, Alberta']],
                         columns = ['Letter', 'City'])
city_codes

,Letter,City
0,M,"Toronto, Ontario"
1,H,"Montreal, Quebec"
2,T,"Calgary, Alberta"


In [3]:
fsa_set=[]

for url in ('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_' + letter for letter in city_codes.iloc[:,0]):

    html = requests.get(url).content
    assert html[:15].decode("utf-8").lower()=='<!doctype html>', 'HTML required' # Check for html format, if not raise error
    df = pd.read_html(html)[0] #read html table as df 
    
    # 'M' postal code table is usable as is, other tables need to be reshaped.
    
    if 'Postcode' not in df.columns: # grid without headers
        pc_list = []
        for c in df.columns:
            pc_list.extend(list(df[c])) # stack columns
            
        fsa = [r[:3] for r in pc_list] # split strings (first 3 letters)
        descr = [r[3:] for r in pc_list] # split strings (the rest)
        df = pd.DataFrame(zip(fsa,descr)) # recreate dataframe

        if url[-1] == 'T': # T's include all Alberta. Exclude all but Calgary
            df = df[df.iloc[:,1].str.contains('Calgary', case=False)]
            
    df = df[df.iloc[:,1].str.contains('not assigned', case=False)==False] # Exclude not assigned
    
    fsa_set.extend(df.iloc[:,0]) # add to list of fsa

fsa_set = sorted(set(fsa_set)) # sorted list of unique fsa

print('{} forward sortation areas found.'.format(len(fsa_set)))
fsa_set[0:5]

419 forward sortation areas found.


['Agi', 'Ald', 'Bat', 'Bay', 'Bed']

## Use a function to call for the Foursquare API with a search of the closest venues within a radius of each postal code.

Here I will focus only on the food category venues, which is one of the most common and diverse categories on Foursquare. For simplicity I will only look at the primary category of each venue, as some venues have multiple categorizations.

The Foursquare Search endpoint returns the nearest venues to the point of origin, matching the query. I can also add a maximum radius in meters and a maximum number of returned venues.

In [5]:
# function that gets foursquare SEARCH results
def search_venues_fx(places, radius=500, limit=50, categoryId='4d4b7105d754a06374d81259'): 
# max radius=100,000; max limit=50; categoryID: food='4d4b7105d754a06374d81259'
    
    venues_list=[]
    i=0 # counter
    for place in places:
        print(i, place)
        # create the API request URL
        url = ('https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}'
               '&near={}&radius={}&limit={}&categoryId={}').format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                place, 
                radius, 
                limit,
                categoryId)

        # make the GET request
        try: 
            response = requests.get(url).json()['response']
            response['venues'] # check for venues
            coords = response['geocode']['feature']['geometry']['center'] # return geo coords
        except:
            continue # if no results, skip to the next
               
        # return relevant information for each venue and add to list
        venues_list.append([(
            place[0:3], #FSA only
            coords['lat'],
            coords['lng'], 
            v['id'], 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'], 
            v['categories'][0]['name'],
            v['categories'][0]['id'] # very few venues have multiple categories, get only the first
            ) for v in response['venues']])
        
        i+=1 # count successful queries

    # create dataframe from venue list
    df = pd.DataFrame(venue for place_venues in venues_list for venue in place_venues)
    df.columns = [
        'Place', 
        'Place Latitude', 
        'Place Longitude', 
        'Id',
        'Name', 
        'Latitude', 
        'Longitude', 
        'Category',
        'Category Id'
        ]
    
    print('{} places queried. {} venues found.'.format(i, len(df)))
    return(df)

In [6]:
# sample of FSAs for testing
import random
# fsa_sample = random.sample([x for x in fsa_set if x.startswith('M')],k=10)
fsa_sample = random.sample(fsa_set,k=30)
fsa_sample

['T2T',
 'Reg',
 'M2A',
 'T2H',
 'Old',
 'H3A',
 'H7H',
 'M8L',
 'T2Y',
 'M7C',
 'H9X',
 'H7G',
 'M5P',
 'T3G',
 'Yor',
 'Ros',
 'Wob',
 'T2G',
 'M7B',
 'H4P',
 'H2Y',
 'M3K',
 'M4P',
 'M2J',
 'M4L',
 'Dav',
 'H8Z',
 'H3C',
 'M6T',
 'Sca']

In [7]:
t1 = time.perf_counter()

results = search_venues_fx( places=(pc+', Canada' for pc in fsa_set), radius=1000, limit=50)

t2 = time.perf_counter()
print('{:0.2f}s elapsed.'.format(t2-t1))

0 Agi, Canada
0 Ald, Canada
0 Bat, Canada
0 Bay, Canada
0 Bed, Canada
0 Ber, Canada
0 Bir, Canada
0 Bro, Canada
0 Bus, Canada
0 CN , Canada
0 Cal, Canada
0 Can, Canada
0 Ced, Canada
0 Cen, Canada
0 Chr, Canada
0 Chu, Canada
0 Cla, Canada
0 Cli, Canada
0 Com, Canada
0 Dav, Canada
0 Del, Canada
0 Don, Canada
0 Dor, Canada
0 Dow, Canada
0 Duf, Canada
0 Eas, Canada
0 Eri, Canada
0 Eto, Canada
0 Fai, Canada
0 Fir, Canada
0 For, Canada
0 Gar, Canada
0 Gle, Canada
0 Gol, Canada
0 Gui, Canada
0 H0H, Canada
1 H0M, Canada
2 H1A, Canada
3 H1B, Canada
4 H1C, Canada
5 H1E, Canada
5 H1G, Canada
6 H1H, Canada
7 H1J, Canada
8 H1K, Canada
9 H1L, Canada
10 H1M, Canada
11 H1N, Canada
12 H1P, Canada
13 H1R, Canada
14 H1S, Canada
15 H1T, Canada
16 H1V, Canada
17 H1W, Canada
18 H1X, Canada
19 H1Y, Canada
20 H1Z, Canada
21 H2A, Canada
22 H2B, Canada
23 H2C, Canada
24 H2E, Canada
25 H2G, Canada
26 H2H, Canada
27 H2J, Canada
28 H2K, Canada
29 H2L, Canada
30 H2M, Canada
31 H2N, Canada
32 H2P, Canada
33 H2R, Can

Now I ran a function that searches our postal codes in the format 'M1A, Canada'. This is sufficient for Foursquare to geocode the postal code and return the venues, as well as the coordinates for our later use.
I've set the radius to 1km, which looks like more than enough to cover for the area of most postal codes in Toronto. I've also set the limit to 50 results each, which is the documented maximum. I think this should provide a reasonable representation of the majority of food venues throughout the city.

### Checking the Results in a DataFrame

In [8]:
print(results.shape)
results.head()

(8795, 9)


,Place,Place Latitude,Place Longitude,Id,Name,Latitude,Longitude,Category,Category Id
0,H0M,45.6986,-73.5025,4c12f4a3a5eb76b0d93abfb7,Tenuta,45.694548,-73.509593,Italian Restaurant,4bf58dd8d48988d110941735
1,H0M,45.6986,-73.5025,4eee543d93add02fcd2d9b43,Tim Hortons,45.690808,-73.496677,Coffee Shop,4bf58dd8d48988d1e0931735
2,H0M,45.6986,-73.5025,4bf884a25efe2d7fb3f86a34,Dairy Queen,45.696294,-73.491945,Ice Cream Shop,4bf58dd8d48988d1c9941735
3,H0M,45.6986,-73.5025,4e9936fde5fa393c4e31bf42,McDonald's,45.691514,-73.494216,Fast Food Restaurant,4bf58dd8d48988d16e941735
4,H1A,45.6753,-73.5016,5e541e9ee5b4f30008f3c30c,Nat Epicerie,45.668163,-73.499480,Deli / Bodega,4bf58dd8d48988d146941735


### Adding the Cities

In [9]:
# merge venues to cities using the first letter of postal code
place_venues = results
place_venues['Letter'] = [p[0] for p in place_venues['Place']] # create a column for the first letter of the fsa
place_venues = pd.merge(city_codes[['Letter', 'City']], place_venues, left_on='Letter', right_on='Letter') # merge on the letter
place_venues.drop('Letter', axis=1, inplace=True) # drop the letter column

print(place_venues.shape)
place_venues.head()

(8795, 10)


,City,Place,Place Latitude,Place Longitude,Id,Name,Latitude,Longitude,Category,Category Id
0,"Toronto, Ontario",M1B,43.8113,-79.193,517dcdb6f1363b7a770a8424,Meena's Fine Foods,43.804476,-79.199753,Indian Restaurant,4bf58dd8d48988d10f941735
1,"Toronto, Ontario",M1B,43.8113,-79.193,4bb6b9446edc76b0d771311c,Wendy’s,43.807448,-79.199056,Fast Food Restaurant,4bf58dd8d48988d16e941735
2,"Toronto, Ontario",M1B,43.8113,-79.193,4c069a410ed3c9285848787d,Twiga Snack Bar,43.819914,-79.186483,African Restaurant,4bf58dd8d48988d1c8941735
3,"Toronto, Ontario",M1B,43.8113,-79.193,5d013e146a5950002cad73ea,Pizza Pizza,43.819633,-79.184681,Pizza Place,4bf58dd8d48988d1ca941735
4,"Toronto, Ontario",M1B,43.8113,-79.193,5914a5d16a8d866b54e07aed,Dairy Queen,43.818728,-79.185368,Ice Cream Shop,4bf58dd8d48988d1c9941735


### Separate just the postal code coordinates.

In [11]:
# create a table of just postcode coordinates
places_coords = place_venues.iloc[:,:4].drop_duplicates()
places_coords.reset_index(drop=True, inplace=True)

print(places_coords.shape)
places_coords.head()

(235, 4)


,City,Place,Place Latitude,Place Longitude
0,"Toronto, Ontario",M1B,43.8113,-79.1930
1,"Toronto, Ontario",M1C,43.7878,-79.1564
2,"Toronto, Ontario",M1E,43.7678,-79.1866
3,"Toronto, Ontario",M1G,43.7712,-79.2144
4,"Toronto, Ontario",M1H,43.7686,-79.2389


Explore the venues data.
Let's check how many venues were returned for each postal code.

In [12]:
place_venues[['City', 'Place','Name']].groupby(['City','Place']).count().head(10)

Name
City             Place      
Calgary, Alberta T1Y      31
                 T2A      29
                 T2B      10
                 T2C      23
                 T2E      26
                 T2G      41
                 T2H      48
                 T2K      50
                 T2N      31
                 T2R      50

## Finding unique Categories

In [13]:
print('There are {} unique categories across {} venues.'.format(
    len(place_venues['Category'].unique()), 
    place_venues.shape[0]))

There are 195 unique categories across 8795 venues.


In [14]:
place_venues['Category'].value_counts().head(10)

Coffee Shop             986
Café                    561
Pizza Place             540
Restaurant              509
Fast Food Restaurant    414
Bakery                  374
Italian Restaurant      297
Sandwich Place          275
Ice Cream Shop          228
Breakfast Spot          221
Name: Category, dtype: int64

## Going to remove the Coffee Shop
There are large amounts of data for coffee shop category

In [15]:
# remove coffee shop
place_venues = place_venues[(place_venues['Category']!='Coffee Shop')]

print(place_venues.shape)
place_venues['Category'].value_counts().head(10)

(7809, 10)


Café                    561
Pizza Place             540
Restaurant              509
Fast Food Restaurant    414
Bakery                  374
Italian Restaurant      297
Sandwich Place          275
Ice Cream Shop          228
Breakfast Spot          221
Chinese Restaurant      207
Name: Category, dtype: int64


Add in the higher level categories from Foursquare.

In [16]:
# get categories json
url= ('https://api.foursquare.com/v2/venues/categories?&client_id={}&client_secret={}&v={}'
      .format( CLIENT_ID, CLIENT_SECRET, VERSION))
response = requests.get(url).json()['response']

In [17]:
# create a dataframe of categories from the json

# loop through each level of hierarchy, drilling down to get all subcategories
categories_list = []
for c1 in response['categories']:
    if c1['name'].lower()!='food': # food category only
        continue
        
    else:
        # if no subcategories, copy category across all subsequent subcategories
        categories_list.append((
            c1['id'], c1['name'],
            c1['id'], c1['name'],
            c1['id'], c1['name'],
            c1['id'], c1['name'],
            c1['id'], c1['name']))
            
        if len( c1['categories']) > 0: # if subcategories exist, go to next level
            for c2 in c1['categories']:
                categories_list.append((
                    c1['id'], c1['name'],
                    c2['id'], c2['name'],
                    c2['id'], c2['name'],
                    c2['id'], c2['name'],
                    c2['id'], c2['name']))
                    
                if len( c2['categories']) > 0:                    
                    for c3 in c2['categories']:
                        categories_list.append((
                            c1['id'], c1['name'],
                            c2['id'], c2['name'],
                            c3['id'], c3['name'],
                            c3['id'], c3['name'],
                            c3['id'], c3['name']))
                            
                        if len( c3['categories']) > 0:
                            for c4 in c3['categories']:
                                categories_list.append((
                                    c1['id'], c1['name'],
                                    c2['id'], c2['name'],
                                    c3['id'], c3['name'],
                                    c4['id'], c4['name'],
                                    c4['id'], c4['name']))
                                    
                                if len( c4['categories']) > 0:
                                    for c5 in c4['categories']:
                                        categories_list.append((
                                            c1['id'], c1['name'],
                                            c2['id'], c2['name'],
                                            c3['id'], c3['name'],
                                            c4['id'], c4['name'],
                                            c5['id'], c5['name']))

# create dataframe of hierarchy levels as numbered columns, one row per category or subcategory
categories_wide = pd.DataFrame(categories_list)
categories_wide.columns = [
    'id1',
    'name1',
    'id2',
    'name2',
    'id3',
    'name3',
    'id4',
    'name4',
    'id5',
    'name5']

print(categories_wide.shape)
categories_wide.head()

(358, 10)


,id1,name1,id2,name2,id3,name3,id4,name4,id5,name5
0,4d4b7105d754a06374d81259,Food,4d4b7105d754a06374d81259,Food,4d4b7105d754a06374d81259,Food,4d4b7105d754a06374d81259,Food,4d4b7105d754a06374d81259,Food
1,4d4b7105d754a06374d81259,Food,503288ae91d4c4b30a586d67,Afghan Restaurant,503288ae91d4c4b30a586d67,Afghan Restaurant,503288ae91d4c4b30a586d67,Afghan Restaurant,503288ae91d4c4b30a586d67,Afghan Restaurant
2,4d4b7105d754a06374d81259,Food,4bf58dd8d48988d1c8941735,African Restaurant,4bf58dd8d48988d1c8941735,African Restaurant,4bf58dd8d48988d1c8941735,African Restaurant,4bf58dd8d48988d1c8941735,African Restaurant
3,4d4b7105d754a06374d81259,Food,4bf58dd8d48988d1c8941735,African Restaurant,4bf58dd8d48988d10a941735,Ethiopian Restaurant,4bf58dd8d48988d10a941735,Ethiopian Restaurant,4bf58dd8d48988d10a941735,Ethiopian Restaurant
4,4d4b7105d754a06374d81259,Food,4bf58dd8d48988d1c8941735,African Restaurant,5f2c344a5b4c177b9a6dc011,Mauritian Restaurant,5f2c344a5b4c177b9a6dc011,Mauritian Restaurant,5f2c344a5b4c177b9a6dc011,Mauritian Restaurant


In [18]:
# pivot the wide table so that all the category levels become additional rows in the same column
categories = pd.concat([categories_wide[['name2','name3','name4','name5']]], axis=1)
categories = categories.stack()
categories.index = categories.index.droplevel(1) #remove index level created by stack
categories.name = 'categories'
categories = pd.merge(categories_wide['id5'], categories, left_index=True, right_index=True) #merge on index
categories.drop_duplicates(inplace=True)
categories.columns = ['Category Id', 'Categories']

print(categories.shape)
categories.head()

(732, 2)


,Category Id,Categories
0,4d4b7105d754a06374d81259,Food
1,503288ae91d4c4b30a586d67,Afghan Restaurant
2,4bf58dd8d48988d1c8941735,African Restaurant
3,4bf58dd8d48988d10a941735,African Restaurant
3,4bf58dd8d48988d10a941735,Ethiopian Restaurant


### Join the higher level categories to the venues data.
Create a separate table for just unique venues.

In [19]:
# new dataframe of venues only
venues = place_venues.iloc[:,4:].drop_duplicates()

print('{} duplicate venues dropped. {} venues remain.'.format( len(place_venues)-len(venues), len(venues)))
venues.head()

1422 duplicate venues dropped. 6387 venues remain.


,Id,Name,Latitude,Longitude,Category,Category Id
0,517dcdb6f1363b7a770a8424,Meena's Fine Foods,43.804476,-79.199753,Indian Restaurant,4bf58dd8d48988d10f941735
1,4bb6b9446edc76b0d771311c,Wendy’s,43.807448,-79.199056,Fast Food Restaurant,4bf58dd8d48988d16e941735
2,4c069a410ed3c9285848787d,Twiga Snack Bar,43.819914,-79.186483,African Restaurant,4bf58dd8d48988d1c8941735
3,5d013e146a5950002cad73ea,Pizza Pizza,43.819633,-79.184681,Pizza Place,4bf58dd8d48988d1ca941735
4,5914a5d16a8d866b54e07aed,Dairy Queen,43.818728,-79.185368,Ice Cream Shop,4bf58dd8d48988d1c9941735


After merging we should have more rows now that the extra category entries are added for each venue. We may also lose some rows if any venues our search didn't have a food category as their primary category

In [20]:
venue_categories = pd.merge(venues, categories, on='Category Id') # merge on category Id

print(venue_categories.shape)
venue_categories.head()

(7535, 7)


,Id,Name,Latitude,Longitude,Category,Category Id,Categories
0,517dcdb6f1363b7a770a8424,Meena's Fine Foods,43.804476,-79.199753,Indian Restaurant,4bf58dd8d48988d10f941735,Indian Restaurant
1,56c27e75cd10dd4574dbe02a,Hakka No.1,43.772340,-79.185480,Indian Restaurant,4bf58dd8d48988d10f941735,Indian Restaurant
2,4ef38984d5fb0c021e0fcac3,Indias Flavor,43.766228,-79.191298,Indian Restaurant,4bf58dd8d48988d10f941735,Indian Restaurant
3,5065d8afe4b0a44a76c13445,Shalimar Sweets & Samosa,43.762931,-79.209521,Indian Restaurant,4bf58dd8d48988d10f941735,Indian Restaurant
4,4fc6c295e4b0c5a82f0465f8,Al-Hamd Biryani & Pizza,43.767585,-79.219570,Indian Restaurant,4bf58dd8d48988d10f941735,Indian Restaurant


# Analyze Categories
### Use one hot encoding to quantify categorical data, including venues with multiple categories.

Using one hot encoding to count the categories that each venue belongs to. I then group by venue id, so we're back to one row per venue, but the extra categories included using max().

We should end up with slightly less than our original count of venues due to some mismatching categories.

In [21]:
# one hot encoding categories
venues_onehot = pd.get_dummies(venue_categories[['Categories']])

# add id back to dataframe
venues_onehot = pd.merge(venue_categories['Id'], venues_onehot, left_index=True, right_index=True) #merge on index
venues_onehot = venues_onehot.groupby('Id').max().reset_index()

print(venues_onehot.shape)
venues_onehot.head()

(6117, 145)


,Id,Categories_Afghan Restaurant,Categories_African Restaurant,Categories_American Restaurant,Categories_Arepa Restaurant,Categories_Asian Restaurant,Categories_Australian Restaurant,Categories_BBQ Joint,Categories_Bagel Shop,Categories_Bakery,...,Categories_Theme Restaurant,Categories_Tibetan Restaurant,Categories_Turkish Restaurant,Categories_Udon Restaurant,Categories_Vegetarian / Vegan Restaurant,Categories_Venezuelan Restaurant,Categories_Vietnamese Restaurant,Categories_Wings Joint,Categories_Xinjiang Restaurant,Categories_Yemeni Restaurant
0,4ab2b0b9f964a520e56b20e3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4ad4c05cf964a5200ff620e3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,4ad4c05cf964a520b8f520e3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4ad4c05cf964a520cbf520e3,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4ad4c05cf964a520ccf520e3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Check the final list of venues.

In [22]:
# merge final venue ids back to dataframe with latitude, longitude
venues_kept = pd.merge(venues_onehot['Id'], place_venues[['Id', 'City', 'Name', 'Latitude', 'Longitude', 'Category']], 
                    left_on='Id', right_on='Id')
venues_kept.drop_duplicates(inplace=True) # drop duplicates
venues_kept.reset_index(drop=True, inplace=True)

print(venues_kept.shape)
venues_kept.head()

(6117, 6)


,Id,City,Name,Latitude,Longitude,Category
0,4ab2b0b9f964a520e56b20e3,"Toronto, Ontario",The Queen and Beaver Public House,43.657472,-79.383524,Gastropub
1,4ad4c05cf964a5200ff620e3,"Toronto, Ontario",Fresh On Spadina,43.648048,-79.396008,Vegetarian / Vegan Restaurant
2,4ad4c05cf964a520b8f520e3,"Toronto, Ontario",The Drake Hotel,43.643147,-79.424597,Restaurant
3,4ad4c05cf964a520cbf520e3,"Toronto, Ontario",Safari Bar and Grill,43.729051,-79.418109,American Restaurant
4,4ad4c05cf964a520ccf520e3,"Toronto, Ontario",Auberge du Pommier,43.746962,-79.407879,French Restaurant


### Group by postal code.

Now I repeat the process but group by FSA and calculating the mean. The mean represents the proportion of venues of each category, out of all the venues near the postal code. This will be our metric used for clustering analysis.

In [23]:
# add postcode back to dataframe
places_onehot = pd.merge(place_venues[['Place','Id']], venues_onehot, left_on='Id', right_on='Id') #merge on index

# group and calculate proportion of each category
places_mean = places_onehot.groupby('Place').mean().reset_index()

print(places_mean.shape)
places_mean.head()

(235, 145)


,Place,Categories_Afghan Restaurant,Categories_African Restaurant,Categories_American Restaurant,Categories_Arepa Restaurant,Categories_Asian Restaurant,Categories_Australian Restaurant,Categories_BBQ Joint,Categories_Bagel Shop,Categories_Bakery,...,Categories_Theme Restaurant,Categories_Tibetan Restaurant,Categories_Turkish Restaurant,Categories_Udon Restaurant,Categories_Vegetarian / Vegan Restaurant,Categories_Venezuelan Restaurant,Categories_Vietnamese Restaurant,Categories_Wings Joint,Categories_Xinjiang Restaurant,Categories_Yemeni Restaurant
0,H0M,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,H1A,0.0,0.0,0.000000,0.0,0.200000,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,H1B,0.0,0.0,0.000000,0.0,0.090909,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,H1C,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,H1G,0.0,0.0,0.021739,0.0,0.108696,0.0,0.021739,0.0,0.065217,...,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,0.0,0.0,0.0


In [24]:
# group and calculate TOTAL of each category
places_sum = places_onehot.groupby('Place').sum().reset_index()

print(places_sum.shape)
places_sum.head()

(235, 145)


,Place,Categories_Afghan Restaurant,Categories_African Restaurant,Categories_American Restaurant,Categories_Arepa Restaurant,Categories_Asian Restaurant,Categories_Australian Restaurant,Categories_BBQ Joint,Categories_Bagel Shop,Categories_Bakery,...,Categories_Theme Restaurant,Categories_Tibetan Restaurant,Categories_Turkish Restaurant,Categories_Udon Restaurant,Categories_Vegetarian / Vegan Restaurant,Categories_Venezuelan Restaurant,Categories_Vietnamese Restaurant,Categories_Wings Joint,Categories_Xinjiang Restaurant,Categories_Yemeni Restaurant
0,H0M,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,H1A,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,H1B,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,H1C,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,H1G,0,0,1,0,5,0,1,0,3,...,0,0,0,0,0,0,2,0,0,0


# K-Means Cluster

In [25]:
# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans( n_clusters=kclusters, random_state=0).fit( places_mean.iloc[:,1:])

# preview cluster labels
print(len(kmeans.labels_))
print(kmeans.labels_[0:10])

235
[2 2 2 3 4 4 4 4 4 4]


In [26]:
def top_categories_fx(row, num_categories):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_categories]

### The Top Categories

In [27]:
num_categories = 5

# create column names
number_suffixes = ['st', 'nd', 'rd']
columns = ['Place']
for n in np.arange(num_categories):
    if n<len(number_suffixes):
        columns.append('{}{} Most Frequent'.format(n+1, number_suffixes[n]))
    else:
        columns.append('{}th Most Frequent'.format(n+1))

# create a new dataframe
place_top_categories = pd.DataFrame(columns=columns)
place_top_categories['Place'] = places_mean['Place']

for n in np.arange( places_mean.shape[0]):
    place_top_categories.iloc[n, 1:] = top_categories_fx( places_mean.iloc[n, :], num_categories)

print(place_top_categories.shape)
place_top_categories.head()

(235, 6)


,Place,1st Most Frequent,2nd Most Frequent,3rd Most Frequent,4th Most Frequent,5th Most Frequent
0,H0M,Categories_Italian Restaurant,Categories_Fast Food Restaurant,Categories_Dessert Shop,Categories_Ice Cream Shop,Categories_Food Stand
1,H1A,Categories_Pizza Place,Categories_Italian Restaurant,Categories_Asian Restaurant,Categories_Deli / Bodega,Categories_Chinese Restaurant
2,H1B,Categories_Restaurant,Categories_Pizza Place,Categories_Ice Cream Shop,Categories_Japanese Restaurant,Categories_Asian Restaurant
3,H1C,Categories_Bakery,Categories_Fried Chicken Joint,Categories_Food Truck,Categories_Food Stand,Categories_Food Court
4,H1G,Categories_Restaurant,Categories_Fast Food Restaurant,Categories_Asian Restaurant,Categories_Sandwich Place,Categories_Pizza Place


Create summary dataframes.
Combine the Postal Code location, cluster and top categories.

In [29]:
# join to place coords
place_clusters = pd.merge(places_coords, place_top_categories, left_on='Place', right_on='Place')
# add cluster labels
place_clusters.insert(4, 'Cluster', kmeans.labels_)
place_clusters.sort_values(by=['Cluster'], inplace=True) # sort by cluster

print(place_clusters.shape)
place_clusters.head()

(235, 10)


,City,Place,Place Latitude,Place Longitude,Cluster,1st Most Frequent,2nd Most Frequent,3rd Most Frequent,4th Most Frequent,5th Most Frequent
117,"Montreal, Quebec",H1Y,45.5486,-73.5788,0,Categories_Café,Categories_Asian Restaurant,Categories_Italian Restaurant,Categories_Bakery,Categories_Restaurant
141,"Montreal, Quebec",H3E,45.4594,-73.5501,0,Categories_Pizza Place,Categories_Dessert Shop,Categories_Asian Restaurant,Categories_Italian Restaurant,Categories_Ice Cream Shop
132,"Montreal, Quebec",H2S,45.5354,-73.6061,0,Categories_Café,Categories_Pizza Place,Categories_Asian Restaurant,Categories_Fast Food Restaurant,Categories_Bakery
51,"Toronto, Ontario",M4Y,43.6656,-79.3830,0,Categories_Fast Food Restaurant,Categories_Asian Restaurant,Categories_Japanese Restaurant,Categories_Gastropub,Categories_Café
129,"Montreal, Quebec",H2N,45.5394,-73.6513,0,Categories_Asian Restaurant,Categories_Breakfast Spot,Categories_Café,Categories_Japanese Restaurant,Categories_Restaurant


Group by cluster.
One-hot to count the number of places from each city in each cluster.

In [30]:
# one hot encoding cities
cities_onehot = pd.get_dummies(place_clusters['City'])

# add cluster back to dataframe
cities_onehot = pd.merge(place_clusters['Cluster'], cities_onehot, left_index=True, right_index=True) #merge on index
cities_onehot = cities_onehot.groupby('Cluster').sum().reset_index()

print(cities_onehot.shape)
cities_onehot.head()

(5, 4)


,Cluster,"Calgary, Alberta","Montreal, Quebec","Toronto, Ontario"
0,0,5,14,6
1,1,0,2,0
2,2,5,18,11
3,3,0,0,3
4,4,15,77,79


Create another summary dataframe combiing the city counts with the most frequent categories, but by cluster.

In [31]:
cluster_sum = place_clusters[['Cluster','Place']]

# join to category sums
cluster_sum = pd.merge(cluster_sum, places_sum, left_on='Place', right_on='Place')
cluster_sum.drop(['Place'], axis=1)

cluster_mean = cluster_sum.groupby('Cluster').mean().reset_index()

print(cluster_mean.shape)
cluster_mean.head()

(5, 145)


,Cluster,Categories_Afghan Restaurant,Categories_African Restaurant,Categories_American Restaurant,Categories_Arepa Restaurant,Categories_Asian Restaurant,Categories_Australian Restaurant,Categories_BBQ Joint,Categories_Bagel Shop,Categories_Bakery,...,Categories_Theme Restaurant,Categories_Tibetan Restaurant,Categories_Turkish Restaurant,Categories_Udon Restaurant,Categories_Vegetarian / Vegan Restaurant,Categories_Venezuelan Restaurant,Categories_Vietnamese Restaurant,Categories_Wings Joint,Categories_Xinjiang Restaurant,Categories_Yemeni Restaurant
0,0,0.120000,0.000000,0.400000,0.04,5.280000,0.000000,0.200000,0.160000,1.840000,...,0.040000,0.00000,0.040000,0.000000,0.280000,0.000000,1.160000,0.000000,0.000000,0.000000
1,1,0.000000,0.500000,0.500000,0.00,8.000000,0.000000,1.000000,0.000000,1.500000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
2,2,0.029412,0.205882,0.382353,0.00,3.911765,0.000000,0.205882,0.176471,1.617647,...,0.029412,0.00000,0.058824,0.000000,0.058824,0.000000,0.529412,0.058824,0.000000,0.000000
3,3,0.000000,0.000000,0.666667,0.00,7.000000,0.000000,0.000000,0.000000,1.333333,...,0.000000,0.00000,0.333333,0.000000,0.333333,0.000000,0.333333,0.000000,0.000000,0.000000
4,4,0.052632,0.093567,0.479532,0.00,4.847953,0.005848,0.380117,0.251462,1.555556,...,0.005848,0.02924,0.093567,0.005848,0.251462,0.017544,0.608187,0.128655,0.005848,0.005848


In [32]:
num_categories = 5

# create column names
number_suffixes = ['st', 'nd', 'rd']
columns = ['Cluster']
for n in np.arange(num_categories):
    if n<len(number_suffixes):
        columns.append('{}{} Most Frequent'.format(n+1, number_suffixes[n]))
    else:
        columns.append('{}th Most Frequent'.format(n+1))

# create a new dataframe
cluster_top_categories = pd.DataFrame(columns=columns)
cluster_top_categories['Cluster'] = cluster_mean['Cluster']

for n in np.arange( cluster_mean.shape[0]):
    cluster_top_categories.iloc[n, 1:] = top_categories_fx( cluster_mean.iloc[n, :], num_categories)

cluster_summary = pd.merge(cities_onehot, cluster_top_categories, left_on='Cluster', right_on='Cluster')
    
print(cluster_summary.shape)
cluster_summary.head()


(5, 9)


,Cluster,"Calgary, Alberta","Montreal, Quebec","Toronto, Ontario",1st Most Frequent,2nd Most Frequent,3rd Most Frequent,4th Most Frequent,5th Most Frequent
0,0,5,14,6,Categories_Asian Restaurant,Categories_Café,Categories_Restaurant,Categories_Fast Food Restaurant,Categories_Dessert Shop
1,1,0,2,0,Categories_Asian Restaurant,Categories_Fast Food Restaurant,Categories_Sandwich Place,Categories_Pizza Place,Categories_Café
2,2,5,18,11,Categories_Asian Restaurant,Categories_Pizza Place,Categories_Café,Categories_Restaurant,Categories_Bakery
3,3,0,0,3,Categories_Asian Restaurant,Categories_Korean Restaurant,Categories_Café,Categories_Dessert Shop,Categories_Italian Restaurant
4,4,15,77,79,Categories_Asian Restaurant,Categories_Café,Categories_Pizza Place,Categories_Restaurant,Categories_Dessert Shop


We can see that Asian restaurants are the most frequent across all clusters, followed by pizza places, fast food and the overall restaurant category. However once we get into the 4th and 5th most frequent categories differences in clusters emerge.

### Group by city.

As above, but by city.

In [33]:
# one hot encoding clusters
clusters_onehot = pd.get_dummies(place_clusters['Cluster'], prefix="Cluster", prefix_sep="")

# add city back to onehot of clusters and group by city
clusters_onehot = pd.merge(place_clusters['City'], clusters_onehot, left_index=True, right_index=True) #merge on index
clusters_onehot = clusters_onehot.groupby('City').sum().reset_index()


# join city to category 
city_sum = place_clusters[['City','Place']]
city_sum = pd.merge(city_sum, places_sum, left_on='Place', right_on='Place')
city_sum.drop(['Place'], axis=1)

# group by city, calculating grand total venues of each category per city
city_mean = city_sum.groupby('City').mean().reset_index()

print(city_mean.shape)
city_mean.head()

(3, 145)


,City,Categories_Afghan Restaurant,Categories_African Restaurant,Categories_American Restaurant,Categories_Arepa Restaurant,Categories_Asian Restaurant,Categories_Australian Restaurant,Categories_BBQ Joint,Categories_Bagel Shop,Categories_Bakery,...,Categories_Theme Restaurant,Categories_Tibetan Restaurant,Categories_Turkish Restaurant,Categories_Udon Restaurant,Categories_Vegetarian / Vegan Restaurant,Categories_Venezuelan Restaurant,Categories_Vietnamese Restaurant,Categories_Wings Joint,Categories_Xinjiang Restaurant,Categories_Yemeni Restaurant
0,"Calgary, Alberta",0.000000,0.000000,0.720000,0.000000,4.560000,0.000000,0.360000,0.120000,0.960000,...,0.000000,0.000000,0.000000,0.000000,0.080000,0.040000,1.200000,0.080000,0.000000,0.04
1,"Montreal, Quebec",0.027027,0.036036,0.207207,0.000000,4.000000,0.009009,0.288288,0.261261,1.684685,...,0.009009,0.009009,0.045045,0.000000,0.261261,0.000000,0.666667,0.018018,0.000000,0.00
2,"Toronto, Ontario",0.101010,0.202020,0.676768,0.010101,5.787879,0.000000,0.383838,0.212121,1.646465,...,0.020202,0.040404,0.151515,0.010101,0.222222,0.020202,0.505051,0.202020,0.010101,0.00


In [34]:
# get top categories by city
num_categories = 5

# create column names
number_suffixes = ['st', 'nd', 'rd']
columns = ['City']
for n in np.arange(num_categories):
    if n<len(number_suffixes):
        columns.append('{}{} Most Frequent'.format(n+1, number_suffixes[n]))
    else:
        columns.append('{}th Most Frequent'.format(n+1))

# create a new dataframe of top categories
city_top_categories = pd.DataFrame(columns=columns)
city_top_categories['City'] = city_mean['City']

for n in np.arange( city_mean.shape[0]):
    city_top_categories.iloc[n, 1:] = top_categories_fx( city_mean.iloc[n, :], num_categories)

# merge clusters and top categories
city_summary = pd.merge(clusters_onehot, city_top_categories, left_on='City', right_on='City')
    
print(city_summary.shape)
city_summary.head()

(3, 11)


,City,Cluster0,Cluster1,Cluster2,Cluster3,Cluster4,1st Most Frequent,2nd Most Frequent,3rd Most Frequent,4th Most Frequent,5th Most Frequent
0,"Calgary, Alberta",5,0,5,0,15,Categories_Asian Restaurant,Categories_Pizza Place,Categories_Dessert Shop,Categories_Fast Food Restaurant,Categories_Sandwich Place
1,"Montreal, Quebec",14,2,18,0,77,Categories_Asian Restaurant,Categories_Café,Categories_Restaurant,Categories_Dessert Shop,Categories_Pizza Place
2,"Toronto, Ontario",6,0,11,3,79,Categories_Asian Restaurant,Categories_Pizza Place,Categories_Restaurant,Categories_Café,Categories_Fast Food Restaurant


# Mapping

Use geopy library to get the latitude and longitude values of the city.

In [35]:
geolocator = Nominatim(user_agent="explorer") #define a user_agent.

latitude=[]
longitude=[]
for city in city_codes['City']: # loop through cities list
    location = geolocator.geocode(city) # geocode query
    latitude.append(location.latitude)
    longitude.append(location.longitude)

# add to cities dataframe
city_codes['Latitude']=latitude
city_codes['Longitude']=longitude
city_codes

,Letter,City,Latitude,Longitude
0,M,"Toronto, Ontario",43.653482,-79.383935
1,H,"Montreal, Quebec",45.497216,-73.610364
2,T,"Calgary, Alberta",51.053423,-114.062589


In [36]:
for city, c_lat, c_lng in zip(city_codes['City'], city_codes['Latitude'], city_codes['Longitude']):
    
    marker_data = zip(
        places_coords[places_coords['City']==city]['Place'],
        places_coords[places_coords['City']==city]['Place Latitude'],
        places_coords[places_coords['City']==city]['Place Longitude'])
    
    # create map using latitude and longitude values
    map_pc = folium.Map(location=[c_lat, c_lng], zoom_start=11)

    # add markers to map
    for postcode, p_lat, p_lng in marker_data:

        label = postcode
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [p_lat, p_lng],
            radius=5,
            popup=label,
            fill=True,
            fill_opacity=0.7,
            parse_html=False).add_to(map_pc)  

    display(map_pc)

# Visualize Clusters

In [37]:
for city, c_lat, c_lng in zip(city_codes['City'], city_codes['Latitude'], city_codes['Longitude']):
    
    marker_data = zip(
        place_clusters[place_clusters['City']==city]['Place'],
        place_clusters[place_clusters['City']==city]['Cluster'],
        place_clusters[place_clusters['City']==city]['1st Most Frequent'],
        place_clusters[place_clusters['City']==city]['Place Latitude'],
        place_clusters[place_clusters['City']==city]['Place Longitude'])

    # create map
    map_clusters = folium.Map(location=[c_lat, c_lng], zoom_start=11)

    # set color scheme for the clusters
    k_array = np.arange(kclusters)
    colors_array = cm.Set1(np.linspace(0, 1, 9)) # Set1 color map has 9 colors
    color_map = [colors.rgb2hex(i) for i in colors_array]
    # markers_colors = []

    # add markers to the map

    for place, cluster, category, p_lat, p_lng in marker_data:

        label = folium.Popup( '{}\nCluster {}\n{}.'.format(place, cluster, category), parse_html=True)

        folium.CircleMarker(
            [p_lat, p_lng],
            radius=5,
            popup=label,
            color=color_map[cluster%9],
            fill=True,
            fill_color=color_map[cluster%9],
            fill_opacity=0.7).add_to(map_clusters)

    # make legend in html/css from color map
    legend_str=''
    for item, color in zip(k_array, color_map[:kclusters]):
        legend_str+='<tr><td>Cluster {}</td><td><i class="fa fa-circle" style="color:{}"></i></td>'.format(item, color)
    legend_html = '''
        <div style="
            z-index:9999;
            position: fixed; 
            bottom: 40px; 
            right: 40px; 
            padding: 6px;
            border:1px solid grey; 
            font-size:14px;">
        <b>Legend</b>
        <table>
            {}
        </table>
        </div> '''.format(legend_str)
    map_clusters.get_root().html.add_child(folium.Element(legend_html))

    display(map_clusters)